In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as pyw
import hyperspy.api as hys
import tkinter.filedialog as tkf
from tabulate import tabulate
from sklearn.decomposition import NMF, PCA, FastICA, KernelPCA
import malspy
import nimfa
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
# create a customized colorbar
color_rep = ["black", "red", "green", "blue", "purple", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))
custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

In [ ]:
def data_load(adr, rescale=False, crop=None):
    """
    load a spectrum image
    """
    storage = []
    shape = []
    for i, adr in enumerate(adr):
        temp = hys.load(adr)
        temp = temp.isig[crop[0]:crop[1]]
        temp = temp.data
        if rescale:
            tmp = np.max(temp, axis=2)
            temp = temp / tmp[:, :, np.newaxis]
        print(np.max(temp), np.min(temp))
        print(temp.shape)

        shape.append(temp.shape)
        storage.append(temp)
    
    shape = np.asarray(shape)
    return storage, shape

In [ ]:
def reshape_coeff(coeffs, new_shape):
    """
    reshape a coefficient matrix to restore the original scanning shapes.
    """
    coeff_reshape = []
    for i in range(len(new_shape)):
        temp = coeffs[:int(new_shape[i, 0]*new_shape[i, 1]), :]
        coeffs = np.delete(coeffs, range(int(new_shape[i, 0]*new_shape[i, 1])), axis=0)
        temp = np.reshape(temp, (new_shape[i, 0], new_shape[i, 1], -1))
        #print(temp.shape)
        coeff_reshape.append(temp)
        
    return coeff_reshape

In [ ]:
file_adr = []

In [ ]:
file_adr.extend(tkf.askopenfilenames())
print(len(file_adr))
print(*file_adr, sep="\n")

In [ ]:
num_img = len(file_adr)
print(num_img)

In [ ]:
# load spectrum images
cr_range = [1.0, 3.56, 0.01] # actual input
data_storage, data_shape = data_load(file_adr, rescale=True, crop=cr_range)
print(len(data_storage))
print(data_shape)

e_range = np.arange(cr_range[0], cr_range[1], cr_range[2])
depth = len(e_range)
print(len(e_range))

In [ ]:
# create the input dataset
dataset_flat = []
for i in range(num_img):
    dataset_flat.extend(data_storage[i].clip(min=1E-5).reshape(-1, depth).tolist())
    
dataset_flat = np.asarray(dataset_flat)
print(dataset_flat.shape)

In [ ]:
total_num = len(dataset_flat)
ri = np.random.choice(total_num, total_num, replace=False)

dataset_input = dataset_flat[ri]

In [ ]:
num_comp = 5

In [ ]:
lsnmf = nimfa.Lsnmf(dataset_input, seed="random_vcol", rank=num_comp)
lsnmf_fit = lsnmf()

In [ ]:
skl_comp_vectors = np.asarray(lsnmf_fit.coef())
print(skl_comp_vectors.shape)

skl_coeffs = np.asarray(lsnmf_fit.basis())
print(skl_coeffs.shape)

# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(skl_coeffs)
coeffs[ri] = skl_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape)
coeffs_reshape = np.asarray(coeffs_reshape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5)) # all loading vectors
for i in range(num_comp):
    ax.plot(e_range, skl_comp_vectors[i], "-", c=color_rep[i], label="loading vector %d"%(i+1))
#ax.grid()
#ax.legend(fontsize="large")
ax.set_xlabel("eV", fontsize=20)
ax.tick_params(axis="both", labelsize=20)
#ax.axes.get_yaxis().set_visible(False)
fig.tight_layout()
plt.show()

In [ ]:
for i in range(num_comp):
    fig, ax = plt.subplots(1, num_img, figsize=(120, 10))
    min_val = np.min(coeffs_reshape[:][:, :, i])
    max_val = np.max(coeffs_reshape[:][:, :, i])
    for j in range(num_img):
        tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], 
                               vmin=min_val, vmax=max_val, cmap="inferno")
        #ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
        ax[j].axis("off")
    fig.tight_layout()
    #fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
plt.show()

In [ ]:
for i in range(num_comp):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    min_val = np.min(coeffs_reshape[:][:, :, i])
    max_val = np.max(coeffs_reshape[:][:, :, i])
    for j in range(1):
        tmp = ax.imshow(np.zeros((10, 10)), 
                               vmin=min_val, vmax=max_val, cmap="inferno")
        #ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
    c_bar = fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
    c_bar.ax.tick_params(labelsize=30)
plt.show()

In [ ]:
lsnmf = nimfa.Lsnmf(dataset_input, seed="random_vcol", rank=num_comp, max_iter=100)
lsnmf_fit = lsnmf()

In [ ]:
skl_comp_vectors = np.asarray(lsnmf_fit.coef())
print(skl_comp_vectors.shape)

skl_coeffs = np.asarray(lsnmf_fit.basis())
print(skl_coeffs.shape)

# convert the coefficient matrix into coefficient maps
coeffs = np.zeros_like(skl_coeffs)
coeffs[ri] = skl_coeffs.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape)
coeffs_reshape = np.asarray(coeffs_reshape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5)) # all loading vectors
for i in range(num_comp):
    ax.plot(e_range, skl_comp_vectors[i], "-", c=color_rep[i], label="loading vector %d"%(i+1))
#ax.grid()
#ax.legend(fontsize="large")
ax.set_xlabel("eV", fontsize=20)
ax.tick_params(axis="both", labelsize=20)
#ax.axes.get_yaxis().set_visible(False)
fig.tight_layout()
plt.show()

In [ ]:
for i in range(num_comp):
    fig, ax = plt.subplots(1, num_img, figsize=(120, 10))
    min_val = np.min(coeffs_reshape[:][:, :, i])
    max_val = np.max(coeffs_reshape[:][:, :, i])
    for j in range(num_img):
        tmp = ax[j].imshow(coeffs_reshape[j][:, :, i], 
                               vmin=min_val, vmax=max_val, cmap="inferno")
        #ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
        ax[j].axis("off")
    fig.tight_layout()
    #fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
plt.show()

In [ ]:
for i in range(num_comp):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    min_val = np.min(coeffs_reshape[:][:, :, i])
    max_val = np.max(coeffs_reshape[:][:, :, i])
    for j in range(1):
        tmp = ax.imshow(np.zeros((10, 10)), 
                               vmin=min_val, vmax=max_val, cmap="inferno")
        #ax[j].set_title("loading vector %d map"%(i+1), fontsize=10)
        ax.axis("off")
    c_bar = fig.colorbar(tmp, cax=fig.add_axes([0.92, 0.15, 0.04, 0.7]))
    c_bar.ax.tick_params(labelsize=30)
plt.show()